In [2]:
from models.models_one_task import AlexNet, vgg
from datasets.ramas import RamasDataset
from constants import *
from torchsummary import summary
import torch
import torch.nn as nn
import skorch
from skorch.helper import predefined_split
from skorch.dataset import Dataset
from skorch.classifier import NeuralNetClassifier
import skorch.callbacks as callbacks
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

In [2]:
ramas_224_train = RamasDataset(RAMAS_PATH_TO_WAVS, 'Ramas224Descrete',
                 spectrogram_shape=224,
                 augmentation=True, padding='repeat', mode='train',  tasks='emotion', type='descrete')
ramas_224_test = RamasDataset(RAMAS_PATH_TO_WAVS, 'Ramas224Descrete',
                 spectrogram_shape=224,
                 augmentation=False, padding='repeat', mode='test',  tasks='emotion', type='descrete')

============= INITIALIZING DATASET Ramas224Descrete_224_train ===============
============================ SUCCESS! =========================
============= INITIALIZING DATASET Ramas224Descrete_224_test ===============
============================ SUCCESS! =========================


In [3]:
train_dataset = ramas_224_train
test_dataset = ramas_224_test
filename = 'AlexNet--{}_augmentation-{}.md'.format(train_dataset.name, str(train_dataset.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=30, 
                                              threshold_mode='rel', lower_is_better=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = AlexNet(num_classes=8)
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=3e-4, max_epochs=300, batch_size=32, train_split=predefined_split(test_dataset), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
        callback_early_stop
    ]
)

In [5]:
net.fit(train_dataset, y=None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.2073        1.9601       0.2264        1.9107     +  46.6378
      2       0.2225        1.8998       0.2869        1.7789     +  46.1199
      3       0.2647        1.8206       0.2623        1.8543        46.0725
      4       0.2673        1.7855       0.2992        1.7316     +  46.0463
      5       0.2855        1.7609       0.3012        1.7265     +  46.1071
      6       0.2924        1.7486       0.3053        1.7133     +  46.1171
      7       0.2916        1.7321       0.3094        1.6991     +  46.0627
      8       0.3006        1.7136       0.3012        1.6932     +  46.0935
      9       0.3057        1.7159       0.3197        1.6835     +  46.0566
     10       0.3096        1.6948       0.3238        1.6952        46.0964
     11       0.3055        1.6936       0.2869        1.7072        46.0293

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=AlexNet(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
    (classifier): Sequential

In [3]:
train_dataset = ramas_224_train
test_dataset = ramas_224_test
filename = 'VGGNet--{}_augmentation-{}.md'.format(train_dataset.name, str(train_dataset.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=30, 
                                              threshold_mode='rel', lower_is_better=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = vgg(type=11, bn=True, num_classes=8)
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=3e-4, max_epochs=300, batch_size=32, train_split=predefined_split(test_dataset), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
        callback_early_stop
    ]
)

In [5]:
net.fit(train_dataset, y=None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.2365        1.9116       0.1742        3.1456     +  67.4569
      2       0.2704        1.7873       0.3166        1.7154     +  67.3093
      3       0.2940        1.7394       0.2859        1.6974     +  67.3468
      4       0.3091        1.7186       0.2213        2.3580        67.5020
      5       0.3209        1.7093       0.2982        1.6944     +  67.5118
      6       0.3278        1.6698       0.2736        2.4418        67.2240
      7       0.3547        1.6653       0.3463        1.6413     +  67.3927
      8       0.3598        1.6426       0.3443        1.6632        67.3277
      9       0.3865        1.6186       0.3811        1.6149     +  67.1798
     10       0.3975        1.5942       0.3166        1.9065        67.3908
     11       0.4044        1.5757       0.3822        1.7001        67.1412

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=VGG(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace=True)
      (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): BatchNorm2d(256, eps=1e-05, mome

In [2]:
ramas_224_train = RamasDataset(RAMAS_PATH_TO_WAVS_BINARY, 'Ramas224Binary',
                 spectrogram_shape=224,
                 augmentation=True, padding='repeat', mode='train',  tasks='emotion', type='binary')
ramas_224_test = RamasDataset(RAMAS_PATH_TO_WAVS_BINARY, 'Ramas224Binary',
                 spectrogram_shape=224,
                 augmentation=False, padding='repeat', mode='test',  tasks='emotion', type='binary')

============= INITIALIZING DATASET Ramas224Binary_224_train ===============
============================ SUCCESS! =========================
============= INITIALIZING DATASET Ramas224Binary_224_test ===============
============================ SUCCESS! =========================


In [4]:
train_dataset = ramas_224_train
test_dataset = ramas_224_test
filename = 'AlexNet--{}_augmentation-{}.md'.format(train_dataset.name, str(train_dataset.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=40, 
                                              threshold_mode='rel', lower_is_better=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
model = AlexNet(num_classes=2)
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=1e-5, max_epochs=300, batch_size=32, train_split=predefined_split(test_dataset), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
#          callback_early_stop
    ]
)

In [6]:
net.fit(train_dataset, y=None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.5766        0.6871       0.6130        0.6682     +  13.6861
      2       0.5766        0.6795       0.6130        0.6627     +  13.4678
      3       0.5766        0.6771       0.6130        0.6640        13.4681
      4       0.5766        0.6755       0.6130        0.6609     +  13.3835
      5       0.5766        0.6737       0.6130        0.6596     +  13.4660
      6       0.5766        0.6668       0.6130        0.6597        13.3721
      7       0.5864        0.6618       0.6027        0.6525     +  13.7686
      8       0.6014        0.6597       0.6027        0.6507     +  13.3899
      9       0.6351        0.6428       0.6130        0.6426     +  13.3924
     10       0.6377        0.6487       0.5685        0.6723        13.4020
     11       0.6218        0.6458       0.6267        0.6401     +  13.3596

     91       0.8636        0.3298       0.7260        0.6000        13.3703
     92       0.8663        0.3303       0.6644        0.7520        13.3655
     93       0.8707        0.3267       0.7500        0.5865        13.5485
     94       0.8751        0.3125       0.7089        0.6190        13.4140
     95       0.8778        0.2919       0.6747        0.7017        13.4709
     96       0.8822        0.2844       0.7432        0.6369        13.4202
     97       0.8787        0.3006       0.7260        0.6394        13.3891
     98       0.8902        0.2834       0.7192        0.6470        13.3785
     99       0.8937        0.2764       0.7158        0.6588        13.4687
    100       0.9070        0.2546       0.7192        0.7333        13.4412
    101       0.8795        0.2832       0.6712        0.7498        13.4216
    102       0.9026        0.2501       0.6986        0.6988        13.4113
    103       0.9035        0.2453       0.7021        0.7202        13.3974

    191       0.9832        0.0614       0.6986        1.3055        13.4106
    192       0.9832        0.0536       0.6986        1.5063        13.3790
    193       0.9805        0.0659       0.6986        1.3428        13.3629
    194       0.9779        0.0673       0.6918        1.4073        13.3983
    195       0.9894        0.0393       0.6747        1.4952        13.4044
    196       0.9796        0.0588       0.7055        1.4508        13.4033
    197       0.9796        0.0569       0.7089        1.3465        13.4064
    198       0.9646        0.0902       0.7089        1.2962        13.3859
    199       0.9805        0.0635       0.7089        1.3729        13.4198
    200       0.9734        0.0595       0.7055        1.3521        13.4626
    201       0.9858        0.0388       0.7089        1.3633        13.4098
    202       0.9814        0.0500       0.7123        1.4189        13.3851
    203       0.9894        0.0351       0.7021        1.4477        13.4148

    296       0.9965        0.0144       0.7089        1.9500        13.4127
    297       0.9876        0.0314       0.6849        1.7543        13.4423
    298       0.9911        0.0208       0.7192        1.8661        13.4264
    299       0.9911        0.0166       0.7055        1.8925        13.4844
    300       0.9876        0.0251       0.7363        1.6800        13.4490


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=AlexNet(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
    (classifier): Sequential

In [7]:
train_dataset = ramas_224_train
test_dataset = ramas_224_test
filename = 'VGGNet--{}_augmentation-{}.md'.format(train_dataset.name, str(train_dataset.augmentation).lower())
best_model_file_path = os.path.join(RESULTS_FOLDER, filename)
callback_train_acc = callbacks.EpochScoring(scoring="accuracy", 
                                            lower_is_better=False, 
                                            on_train=True, 
                                            name='train_acc')
callback_save_best = callbacks.Checkpoint(monitor='valid_loss_best', 
                                          f_params=None, 
                                          f_optimizer=None, 
                                          f_criterion=None, 
                                          f_history=None, 
                                          f_pickle=best_model_file_path,  
                                          event_name='event_cp')
callback_early_stop = callbacks.EarlyStopping(monitor='valid_loss', patience=40, 
                                              threshold_mode='rel', lower_is_better=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
model = vgg(type=11, bn=True, num_classes=2)
net = skorch.classifier.NeuralNetClassifier(
    model, criterion=nn.CrossEntropyLoss, optimizer=torch.optim.Adam,
    lr=1e-5, max_epochs=300, batch_size=32, train_split=predefined_split(test_dataset), 
    device=device, iterator_train__shuffle=True, 
    callbacks=[
        callback_train_acc,
        callback_save_best,
#         callback_early_stop
    ]
)

In [9]:
net.fit(train_dataset, y=None)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.5934        0.6657       0.3870        0.7708     +  19.7789
      2       0.6289        0.6371       0.4932        0.7340     +  19.7696
      3       0.6351        0.6420       0.6918        0.6220     +  19.8091
      4       0.6643        0.6235       0.6267        0.6467        19.7680
      5       0.6838        0.6094       0.6370        0.6381        19.7855
      6       0.6723        0.6084       0.6849        0.6212     +  19.7667
      7       0.6838        0.5877       0.6541        0.6371        19.7666
      8       0.6944        0.5855       0.6747        0.6322        19.7452
      9       0.6856        0.6001       0.5959        0.6920        19.7843
     10       0.6953        0.5684       0.6849        0.6261        19.8093
     11       0.7068        0.5656       0.7192        0.6133     +  19.7572

     94       0.9894        0.0291       0.8014        0.9330        19.7543
     95       0.9938        0.0216       0.7534        1.2130        19.7307
     96       0.9973        0.0115       0.7740        1.0885        19.8439
     97       0.9991        0.0121       0.7603        1.3643        19.7212
     98       0.9982        0.0109       0.7808        1.1702        19.7795
     99       0.9965        0.0147       0.7500        1.2275        19.7323
    100       0.9655        0.0892       0.7295        1.5141        19.8608
    101       0.9876        0.0328       0.7432        1.4437        19.7255
    102       0.9911        0.0277       0.7603        1.3453        19.8034
    103       0.9973        0.0157       0.7705        1.1028        19.7756
    104       0.9947        0.0171       0.7671        1.2047        19.7832
    105       0.9947        0.0193       0.7226        1.3711        19.8732
    106       0.9991        0.0098       0.7534        1.2527        19.8366

    200       0.9991        0.0036       0.7671        1.4391        19.8383
    201       0.9982        0.0065       0.7500        1.5997        19.8040
    202       0.9965        0.0066       0.7500        1.5305        19.8210
    203       0.9973        0.0051       0.7842        1.5855        19.8001
    204       0.9965        0.0078       0.7637        1.5003        19.7926
    205       0.9841        0.0416       0.5205        5.0827        19.8363
    206       0.9885        0.0304       0.7466        1.5637        19.8579
    207       0.9947        0.0135       0.6712        5.1282        19.7431
    208       0.9982        0.0088       0.7466        1.4682        19.8013
    209       0.9947        0.0216       0.7740        1.7113        19.7644
    210       0.9982        0.0062       0.7363        2.3823        19.7829
    211       0.9929        0.0177       0.7466        1.7097        19.7994
    212       0.9903        0.0176       0.7911        1.5676        19.7669

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=VGG(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace=True)
      (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): BatchNorm2d(256, eps=1e-05, mome

In [1]:
import models.models_multi_task as md_multi
from models.multitask_training_session import *
from datasets.iemocap import IemocapDataset
from datasets.ramas import RamasDataset
from constants import *
from torchsummary import summary
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pandas as pd 

In [2]:
ramas_224_train = RamasDataset(RAMAS_PATH_TO_WAVS_BINARY, 'Ramas224BinaryMultiNormal',
                 spectrogram_shape=224,
                 augmentation=True, padding='repeat', mode='train',  tasks='multi', type='binary')
ramas_224_test = RamasDataset(RAMAS_PATH_TO_WAVS_BINARY, 'Ramas224BinaryMultiNormal',
                 spectrogram_shape=224,
                 augmentation=False, padding='repeat', mode='test',  tasks='multi', type='binary')

============= INITIALIZING DATASET Ramas224BinaryMultiNormal_224_train ===============
============================ SUCCESS! =========================
============= INITIALIZING DATASET Ramas224BinaryMultiNormal_224_test ===============
============================ SUCCESS! =========================


In [3]:
train_ds = ramas_224_train
test_ds = ramas_224_test
model = md_multi.vgg(num_emotions=2, num_speakers=12, num_genders=2, type=11, bn=True)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") 
# device = torch.device("cpu") 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)
ts = TrainingSession(name='VGGAverageWeighting',
                      model=model,
                      train_dataset=train_ds,
                      test_dataset=test_ds,
                      criterion=criterion,
                      optimizer=optimizer,
                      num_epochs=300,
                      batch_size=32,
                      device=device,
                     path_to_weights=WEIGHTS_FOLDER,
                     path_to_results=RESULTS_FOLDER,
                    loss_weighter=averaged_sum  # Из файла multitask_training_session.py
                    )

INITIALIZING TRAINING SESSION...
Loaders ready
TRAINING SESSION VGGAverageWeighting__Ramas224BinaryMultiNormal_224_train INITIALIZED
Trying to load checkpoint from file
File not found, starting from scratch...


In [4]:
ts.execute()

=============TRAINING SESSION STARTED AT EPOCH 1=====================
Epoch #1
# Time passed: 8 s
# Epoch losses | emotion = 0.7608 | speaker = 2.4457 | gender = 0.7015 | total = 1.8070 |
# Train accuracies | emotion = 0.4481842338352524 | speaker = 0.15943312666076173 | gender = 0.4765279007971656 |
# Validation process on validation set
# Validation losses | emotion = 0.0081 | speaker = 0.0341 | gender = 0.0114 | total = 0.0254 |
# Validation accuracies | emotion = 0.386986301369863 | speaker = 0.14726027397260275 | gender = 0.4897260273972603 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #2
# Time passed: 8 s
# Epoch losses | emotion = 0.8596 | speaker = 2.3860 | gender = 0.8186 | total = 1.7511 |
# Train accuracies | emotion = 0.4357838795394154 | speaker = 0.18113374667847654 | gender = 0.46722763507528786 |
# Validation process on validation set
# Validation losses | emotion = 0.0139 | speaker = 0.0322 | gender = 0.0160 | total = 0.0240 |
# Validation 

# Validation losses | emotion = 0.0069 | speaker = 0.0315 | gender = 0.0076 | total = 0.0239 |
# Validation accuracies | emotion = 0.6541095890410958 | speaker = 0.613013698630137 | gender = 0.8835616438356164 |
# Saving checkpoint...
# Done and done!
Epoch #15
# Time passed: 8 s
# Epoch losses | emotion = 0.6370 | speaker = 1.1191 | gender = 0.4129 | total = 0.8487 |
# Train accuracies | emotion = 0.5131975199291409 | speaker = 0.3895482728077945 | gender = 0.6047239444936522 |
# Validation process on validation set
# Validation losses | emotion = 0.0058 | speaker = 0.0150 | gender = 0.0085 | total = 0.0113 |
# Validation accuracies | emotion = 0.684931506849315 | speaker = 0.6404109589041096 | gender = 0.9006849315068494 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #16
# Time passed: 8 s
# Epoch losses | emotion = 0.6403 | speaker = 1.0605 | gender = 0.4021 | total = 0.8136 |
# Train accuracies | emotion = 0.52075952170062 | speaker = 0.40716341895482727 

# Validation losses | emotion = 0.0062 | speaker = 0.0106 | gender = 0.0032 | total = 0.0080 |
# Validation accuracies | emotion = 0.7397260273972602 | speaker = 0.773972602739726 | gender = 0.9075342465753424 |
# Saving checkpoint...
# Done and done!
Epoch #29
# Time passed: 8 s
# Epoch losses | emotion = 0.5562 | speaker = 0.6032 | gender = 0.2694 | total = 0.5284 |
# Train accuracies | emotion = 0.59500320698818 | speaker = 0.564643718884579 | gender = 0.7565437830243426 |
# Validation process on validation set
# Validation losses | emotion = 0.0062 | speaker = 0.0210 | gender = 0.0108 | total = 0.0157 |
# Validation accuracies | emotion = 0.7294520547945206 | speaker = 0.7465753424657534 | gender = 0.8801369863013698 |
# Saving checkpoint...
# Done and done!
Epoch #30
# Time passed: 8 s
# Epoch losses | emotion = 0.5528 | speaker = 0.6043 | gender = 0.2637 | total = 0.5273 |
# Train accuracies | emotion = 0.598819013876587 | speaker = 0.5729554177738412 | gender = 0.762680838500147

# Validation losses | emotion = 0.0054 | speaker = 0.0031 | gender = 0.0025 | total = 0.0042 |
# Validation accuracies | emotion = 0.7123287671232876 | speaker = 0.7876712328767124 | gender = 0.9246575342465754 |
# Saving checkpoint...
# Done and done!
Epoch #43
# Time passed: 8 s
# Epoch losses | emotion = 0.5210 | speaker = 0.4118 | gender = 0.2163 | total = 0.4335 |
# Train accuracies | emotion = 0.6394833872329907 | speaker = 0.660699940264074 | gender = 0.8199064823779019 |
# Validation process on validation set
# Validation losses | emotion = 0.0044 | speaker = 0.0080 | gender = 0.0028 | total = 0.0060 |
# Validation accuracies | emotion = 0.708904109589041 | speaker = 0.7671232876712328 | gender = 0.928082191780822 |
# Saving checkpoint...
# Done and done!
Epoch #44
# Time passed: 8 s
# Epoch losses | emotion = 0.5045 | speaker = 0.3900 | gender = 0.2096 | total = 0.4171 |
# Train accuracies | emotion = 0.6421410741605604 | speaker = 0.6660560431596747 | gender = 0.8230332554956

# Validation losses | emotion = 0.0074 | speaker = 0.0053 | gender = 0.0047 | total = 0.0061 |
# Validation accuracies | emotion = 0.7191780821917808 | speaker = 0.821917808219178 | gender = 0.928082191780822 |
# Saving checkpoint...
# Done and done!
Epoch #57
# Time passed: 8 s
# Epoch losses | emotion = 0.4360 | speaker = 0.2943 | gender = 0.1676 | total = 0.3506 |
# Train accuracies | emotion = 0.6734417975851942 | speaker = 0.7222196323403727 | gender = 0.8549251783133653 |
# Validation process on validation set
# Validation losses | emotion = 0.0038 | speaker = 0.0068 | gender = 0.0026 | total = 0.0051 |
# Validation accuracies | emotion = 0.7534246575342466 | speaker = 0.8356164383561644 | gender = 0.9246575342465754 |
# Saving checkpoint...
# Done and done!
Epoch #58
# Time passed: 8 s
# Epoch losses | emotion = 0.4199 | speaker = 0.3157 | gender = 0.1730 | total = 0.3445 |
# Train accuracies | emotion = 0.6756360526556916 | speaker = 0.7255887114016065 | gender = 0.856907241684

# Validation losses | emotion = 0.0031 | speaker = 0.0034 | gender = 0.0008 | total = 0.0030 |
# Validation accuracies | emotion = 0.7294520547945206 | speaker = 0.839041095890411 | gender = 0.952054794520548 |
# Saving checkpoint...
# Done and done!
Epoch #71
# Time passed: 8 s
# Epoch losses | emotion = 0.3368 | speaker = 0.2325 | gender = 0.1335 | total = 0.2757 |
# Train accuracies | emotion = 0.7051485173218229 | speaker = 0.7649920782444891 | gender = 0.8773437792387629 |
# Validation process on validation set
# Validation losses | emotion = 0.0105 | speaker = 0.0072 | gender = 0.0007 | total = 0.0089 |
# Validation accuracies | emotion = 0.7465753424657534 | speaker = 0.8527397260273972 | gender = 0.9417808219178082 |
# Saving checkpoint...
# Done and done!
Epoch #72
# Time passed: 8 s
# Epoch losses | emotion = 0.3075 | speaker = 0.2190 | gender = 0.1437 | total = 0.2539 |
# Train accuracies | emotion = 0.7073983859856313 | speaker = 0.7676409802184825 | gender = 0.878653675819

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/aggr/anaconda3/envs/ryabinov/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/aggr/anaconda3/envs/ryabinov/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/aggr/anaconda3/envs/ryabinov/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/home/aggr/anaconda3/envs/ryabinov/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/home/aggr/anaconda3/envs/ryabinov/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/aggr/anaconda3/envs/ryabinov/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  Fil

KeyboardInterrupt: 